<a href="https://colab.research.google.com/github/jvataidee/DataAnalysis/blob/master/Analise_Financeira_Mercado_Gado_de_Corte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<p align="center"><img src="https://static.wixstatic.com/media/bdd7cb_ce7cb603d09d4263921dda3fb68a395f~mv2.png" height="75px"></p>

**by: [João Ataíde](https://www.joaoataide.com)**
# Análise Mercado Financeira de Gado de Corte


---

<p align="center"><img src="https://static.wixstatic.com/media/bdd7cb_a59ef6dfb8914eea869878888a5d0acd~mv2.jpg"></p>

Saber que a agricultura é nosso pilar da economia isso não novidade, mas como é possível realizar análises financeiras das commodites? como aplicar ciência de dados para esses mercados? como prever tedências de mercado? como o produtor pode controlar uma industria a céu aberto que é sujeita a diversas variáveis exogenas? 

Foi devido a isso que criei este artigo, onde pretendo realizar algumas análises financeiras, mostrando alguns indicadores e técnincas utilizadas no mercado do agro, neste caso a pecuária especificamente a cultura de animais de corte.

## O que são commodities?

- Live Cattle - Boi Gordo
- Feeder Cattle - Bezerro

## Como analisas commodities?


## Importanto bibliotecas básicas

In [1]:
#importar bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# configurar a visualização
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## Obtenção dos dados

Encontrar dados para começar as análises sempre é um processo complicado, em especial quando a gente está falando de um eixo de mercado ainda muito artesanal, o qual etá modificando agora seu modo com a Pecuária 4.0, assim, não foi possível encontrar uma API nacional que automatize a obtenção dados sobre o preço das commodities.

Então, para isso realizei a instalação de duas bibliotecas investpy, API criada a parti da plataforma [investing.com](https://investing.com/) que possui diversos bases de dados financeiras, não somente de commodities. Outra biblioteca que foi necessária foi a [Yahoo! Finance](https://finance.yahoo.com/) a qual utilizei para obter os dados referentes a cotas do dollar.


In [2]:
!pip install investpy -q

     |████████████████████████████████| 4.4 MB 4.9 MB/s 
     |████████████████████████████████| 241 kB 45.6 MB/s 
     |████████████████████████████████| 6.3 MB 38.9 MB/s 
     |████████████████████████████████| 510 kB 42.6 MB/s 


In [3]:
!pip install yfinance -q

In [4]:
pip install -U sidrapy -q

In [5]:
#importar bibliotecas financeiras
import investpy
import yfinance as yf
import sidrapy

In [6]:
#Lista de commodities
commodities = ["Live Cattle", "Feeder Cattle", ]

#Data de inicio e fim de análise
inicio = "01/07/2014"
fim = "01/07/2021"

In [7]:
#Função para importar dados das commodities
def commodities_df(commodities, inicio,fim):
  precos = pd.DataFrame()
  for i in commodities:
    df = investpy.commodities.get_commodity_historical_data(commodity=i, from_date=inicio, to_date=fim).Close
    df.rename(i, inplace=True)
    df.columns = [i]
    precos = pd.concat([precos,df], axis=1)
    precos.index.name="Date"
  return precos

In [8]:
#Data Frame de commodites
df = commodities_df(commodities,inicio,fim)
df.head()

,Live Cattle,Feeder Cattle
Date,,
2014-07-01 00:00:00,151.500,215.78
2014-07-02 00:00:00,152.325,216.20
2014-07-03 00:00:00,155.000,217.62
2014-07-07 00:00:00,154.950,218.00
2014-07-08 00:00:00,153.525,215.90


In [9]:
#Importar historico do dollar
doll = yf.Ticker("USDBRL=x")
dollar = pd.DataFrame(doll.history(start ="2014-07-01", end ="2021-07-01" ).Close)
dollar.columns = ["Dollar"]
dollar.index.name="Date"
dollar.head()

,Dollar
Date,
2014-07-01,2.2117
2014-07-02,2.1981
2014-07-03,2.2223
2014-07-04,2.2068
2014-07-07,2.2098


In [ ]:
https://apisidra.ibge.gov.br/values/t/1092/n1/all/v/all/p/all/c12716/all/c18/55,56,992,111734,111735/c12529/all/d/v1000151%202,v1000284%202,v1000285%202

In [117]:
import pandas as pd
import requests

consulta = 'https://apisidra.ibge.gov.br/values/t/1092/n1/all/v/all/p/all/c12716/all/c18/55,56,992,111734,111735/c12529/all/d/v1000151%202,v1000284%202,v1000285%202'
ufs = [5]

df2 = pd.DataFrame()
for uf in ufs:
    url = consulta.format(uf)
    dados_json = requests.get(url).json()
    dados_pandas = pd.DataFrame(dados_json)
    df2 = pd.concat([dados_pandas,df2])
    
sidra = df2.copy() 
colunas = list(df2.iloc[0,:])
sidra.columns = colunas
sidra = sidra.iloc[1:,:]
sidra = sidra[sidra["Variável"]=="Animais abatidos"]
sidra = sidra[sidra["Tipo de inspeção"]=="Federal"]
sidra = sidra.iloc[:, [4, 10, 14]]
sidra = sidra.drop(sidra[sidra["Valor"] == 'X'].index)
sidra["Valor"] = sidra["Valor"].astype(float)

sidra.head()

,Valor,Trimestre,Tipo de rebanho bovino
15522,1376884.0,1º trimestre 1997,Bois
15526,867851.0,1º trimestre 1997,Vacas
15530,2726534.0,1º trimestre 1997,Total
15534,376902.0,1º trimestre 1997,Novilhos
15538,101791.0,1º trimestre 1997,Novilhas


In [118]:
bois = sidra[sidra["Tipo de rebanho bovino"]=="Bois"]
vacas = sidra[sidra["Tipo de rebanho bovino"]=="Vacas"]
novilhas = sidra[sidra["Tipo de rebanho bovino"]=="Novilhas"]
novilhos = sidra[sidra["Tipo de rebanho bovino"]=="Novilhos"]
totais = sidra[sidra["Tipo de rebanho bovino"]=="Total"]

##Análises exploratória

###Pré-processamento


### Análises da série temporal

## Modelo de previsão

### Tratamento dos dados

##Conclusão